In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re

In [28]:
DATA_DIR = Path("preproc_Datasets/preproc_Datasets/datasets")      # при необходимости поменяйте путь
PATTERN   = "dataset_raw*.csv"            # шаблон имён файлов

names = ["t", "ax", "ay", "yawRate", "steer"]  # 6-я временная

def natural_key(p):                     # сортировка raw1, raw2, …
    parts = re.split(r"(\d+)", p.stem)
    return [int(s) if s.isdigit() else s for s in parts]

dfs_circle = []
for f in sorted(DATA_DIR.glob(PATTERN), key=natural_key):
    df = pd.read_csv(
        f,
        engine="python"     # терпимее к кривым строкам
    )
    dfs_circle.append(df)

In [29]:
DATA_DIR = Path("preproc_Datasets/preproc_Datasets/datasets_slide")      # при необходимости поменяйте путь
PATTERN   = "dataset_raw*.csv"            # шаблон имён файлов

names = ["t", "ax", "ay", "yawRate", "steer"]  # 6-я временная

def natural_key(p):                     # сортировка raw1, raw2, …
    parts = re.split(r"(\d+)", p.stem)
    return [int(s) if s.isdigit() else s for s in parts]

dfs_zanos = []
for f in sorted(DATA_DIR.glob(PATTERN), key=natural_key):
    df = pd.read_csv(
        f,
        engine="python"     # терпимее к кривым строкам
    )
    dfs_zanos.append(df)

In [30]:
DATA_DIR = Path("preproc_Datasets/preproc_Datasets/datasets_straight")      # при необходимости поменяйте путь
PATTERN   = "dataset_raw*.csv"            # шаблон имён файлов

names = ["t", "ax", "ay", "yawRate", "steer"]  # 6-я временная

def natural_key(p):                     # сортировка raw1, raw2, …
    parts = re.split(r"(\d+)", p.stem)
    return [int(s) if s.isdigit() else s for s in parts]

dfs_forward = []
for f in sorted(DATA_DIR.glob(PATTERN), key=natural_key):
    df = pd.read_csv(
        f,
        engine="python"     # терпимее к кривым строкам
    )
    dfs_forward.append(df)

In [31]:
def data_to_world_frame(df):
    # Время в секундах
    df["t_sec"] = df["t"] / 1000
    dt = df["t_sec"].diff().fillna(0.02)  # подставим 20 мс, если NaN

    # Интеграция угла
    yaw = [0]
    for i in range(1, len(df)):
        dyaw = df["yawRate"].iloc[i] * dt.iloc[i]
        yaw.append(yaw[-1] + dyaw)
    df["yaw"] = yaw

    # Перевод угла в радианы
    df["yaw_rad"] = np.deg2rad(df["yaw"])

    # Поворот локального ускорения в мировую систему координат
    ax_world = []
    ay_world = []

    for i in range(len(df)):
        theta = df["yaw_rad"].iloc[i]
        ax = df["ax"].iloc[i]
        ay = df["ay"].iloc[i]

        ax_w = ax * np.cos(theta) - ay * np.sin(theta)
        ay_w = ax * np.sin(theta) + ay * np.cos(theta)

        ax_world.append(ax_w)
        ay_world.append(ay_w)

    df["ax_world"] = ax_world
    df["ay_world"] = ay_world

    # Производная yawRate → угловое ускорение
    df["angular_accel"] = df["yawRate"].diff() / dt
    df["angular_accel"] = df["angular_accel"].fillna(0)

    plt.plot(df["t_sec"], df["angular_accel"], label="Angular Acceleration")
    plt.xlabel("Time (s)")
    plt.legend()
    plt.show()

    return df

In [32]:
!mkdir -p data_in_world_frame/datasets
!mkdir -p data_in_world_frame/datasets_slide
!mkdir -p data_in_world_frame/datasets_straight

ЋиЁЎЄ  ў бЁ­в ЄбЁбҐ Є®¬ ­¤л.
ЋиЁЎЄ  ў бЁ­в ЄбЁбҐ Є®¬ ­¤л.
ЋиЁЎЄ  ў бЁ­в ЄбЁбҐ Є®¬ ­¤л.


In [33]:
n = 1
for df_i in dfs_circle:
    df_new = data_to_world_frame(df_i)
    df_new.to_csv(f'data_in_world_frame/data_in_world_frame/datasets/dataset_raw{n}.csv', index=False)
    n += 1

In [34]:
n = 1
for df_i in dfs_zanos:
    df_new = data_to_world_frame(df_i)
    df_new.to_csv(f'data_in_world_frame/data_in_world_frame/datasets_slide/dataset_raw{n}.csv', index=False)
    n += 1

In [35]:
n = 1
for df_i in dfs_forward:
    df_new = data_to_world_frame(df_i)
    df_new.to_csv(f'data_in_world_frame/data_in_world_frame/datasets_straight/dataset_raw{n}.csv', index=False)
    n += 1

In [36]:
!zip -r data_in_world_frame.zip data_in_world_frame/

"zip" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
